# Python realization and optimization of Biclustering via SSVD

## Abstract

As an approach of approximating large, noisy real-world data matrices, sparse singular value decomposition (SSVD) provides a low-rank, checkerboard structured matrix approximation method. The key idea is to obtain sparse left- and right-singular vectors that contains many zero elements, by inducing regularization penalties to least square regressions. An iterative algorithm using BIC is introduced for the lack of closed form solutions for l1 regularization like Lasso penalty. Optimizations including parallelism and C++ wrapping using pybind11 are performed. A simulated dataset, a lung cancer dataset, and a breast cancer dataset are used for demonstration of biclustering using SSVD. Comparative analysis between Sparse SVD, SVD, and Sparse PCA using simulated dataset is also presented.

##### Key Words: Sparse SVD; PCA; Lasso Penalty; Dimension Reduction

## Background

The research paper we are using is __Biclustering via Sparse Singular Value Decomposition__ by Mihee Lee, Haipeng Shen, Jianhua Z. Huang, and J. S. Marron. This paper provides a new exploratory analysis tool for biclustering or identifying interpretable row-column associations within data matrices. As an unsupervised learning method, biclustering plays an important role in data exploratory analysis, whose goal is to interpret the structure of the data matrices. The biclustering method aims to indentify the patterns in the rows set and columns set simultsneously, and forms an interpretable checkerboard structure of the data matrices. Sparse Singular Value Decomposition(SSVD) as an approach of biclustering mainly aims to solve the problems based on high-dimension low sample size (HDLSS) data.

The biclustering method of HDLSS data is mainly applied in the fields such as, text mining/categorization, biomedical application, and microarray gene expression analysis. In the text mining, SSVD biclustering finds the document and word associations simultaneously and reveals the important relations between document and word clusters. In the field of biomedical application, SSVD biclustering can be used to associate the common properties of chemical compounds in the drug and nutritional data. As for the microarray gene expression analysis, SSVD biclustering allows simultaneously diagnose conditions from sample clusters and identify corresponding genes.

As a comparison to SVD, SSVD performs better in detecting the underlying sparse structure of HDLSS data matrices. In addtion, SSVD is able to detect the sparse structure for both directions simultaneously, while SPAC fails in the unpenalized direction. However, since there is no closed form solution for regression with adaptive lasso penalty, iterative algorithm is required for SSVD method, which results in larger compution cost.

In this paper, we use BIC to choose the degree of sparsity for each round of updates and fix both weights parameters of Lasso penalty to 2, which is suggested by both Zou (2006) and Lee (2010). The resulted SSVD is used to conduct rank-1 approximation for sparse matrices with simulated noise added, and for real life lung cancer and breast cancer datasets.

## Description of algorithm

The biclutersing via sparse singular value decomposition algorithm can be divided into two major parts. Firstly, sparse singular value decomposition (SSVD) provides a rank-k (low-rank) matrix approximation to the data matrix, which is the combination/summation of the top k ranks SSVD layer. Each SSVD layer is obtained from the residual matrix of the previous layer, based on a penalized sum-of-squares criterion. Secondly, the biclustering part includes clustering both the row and column variables based on the left and right singular of the SSVD results respectively. The checkerboard structured matrix approximation is then obtain by sorting the singular vector within each clustering group. 

Without the loss of generality, we only consider the rank-1 matrix approximation via SSVD, which is the first SSVD layer, and the corresponding biclustering method.

##### The SSVD Algorithm

Step 1. (Initialize $u$, $s$, $v$)

Apply the standard SVD to $X$ and obtain the first SVD triplet $u$, $s$, $v$, where $X = usv^T$.

Step 2. (Update $u$, $v$):

(a). update $v$.

For each possible $\lambda_v$:
<br>
1. calculate the corresponding $v$, where $v_j = sign{(X^T u)_j}(|(X^T u)_j|-\lambda_v w_{v,j}/2)_{+}$, $\{j = 1, \cdots, d\}$.
2. calculate the corresponding $BIC$, where $BIC(\lambda_v) = \frac{||Y-\hat{Y}||^2}{nd\cdot \hat{\sigma_v^2}} + \frac{log(nd)}{nd}\hat{df}(\lambda_v)$

Pick the $\lambda_v$ that minimizes $BIC(\lambda_v)$ as the penalty parameter for $v$ and set the corresponding $v$ to be $v_{new}$, the update of the original $v$.
<br>
Rescale $v_{new}$, s.t. $v_{new} = v_{new}/s$, where $s = ||v_{new}||$.

(b). update $u$.

For each possible $\lambda_u$:
<br>
1. calculate the corresponding $u$, where $u_i = sign{(X v_{new})_i}(|(X v_{new})_i|-\lambda_u w_{u,i}/2)_{+}$, $\{i = 1, \cdots, n\}$.
2. calculate the corresponding $BIC$, where $BIC(\lambda_u) = \frac{||Z-\hat{Z}||^2}{nd\cdot \hat{\sigma_u^2}} + \frac{log(nd)}{nd}\hat{df}(\lambda_u)$

Pick the $\lambda_u$ that minimizes $BIC(\lambda_u)$ as the penalty parameter for $u$ and set the corresponding $u$ to be $u_{new}$, the update of the original $u$.
<br>
Rescale $u_{new}$, s.t. $u_{new} = u_{new}/s$, where $s = ||u_{new}||$.

(c). Set $u = u_{new}$ and $v = v_{new}$, repeat Step 2(a) and 2(b) until convergence, where both $\Delta u$ and $\Delta v$ are smaller than the tolerance.

Step 3. return $u$, $s$, $v$ at convergence.

Note: 
1. $\hat{df}(\lambda_v)$ is the degree of sparsity of $v$, which is defined by the number of $|(X^T u)_j|$s thata are greater than $\lambda_v w_{v,j}/2$.
2. $\hat{\sigma_v^2}$ is the OLS estimae of the error variance from the model $||X - u\tilde{v}||_F^2 + \lambda_v P_2(\tilde{v})$, where $\tilde{v} = sv$.
3. $\hat{df}(\lambda_u)$ is the degree of sparsity of $u$, which is defined by the number of $|(X v)_i|$s thata are greater than $\lambda_u w_{u,j}/2$.
4. $\hat{\sigma_u^2}$ is the OLS estimae of the error variance from the model $||X - \tilde{u}v||_F^2 + \lambda_u P_1(\tilde{u})$, where $\tilde{u} = su$.

## Describe optimization for performance

First implement the algorithm using plain Python in a straightforward way from the description of the algorihtm. Then profile and optimize it using one or more apporpiate mathods, such as:

1. Use of better algorithms or data structures
2. Use of vectorization
3. JIT or AOT compilation of critical functions
4. Re-writing critical functions in C++ and using pybind11 to wrap them
5. Making use of parallelism or concurrency
6. Making use of distributed compuitng

Document the improvemnt in performance with the optimizations performed.

## Applications to simulated data sets

Are there specific inputs that give known outuputs (e.g. there might be closed form solutions for special input cases)? How does the algorithm perform on these? 

If no such input cases are available (or in addition to such input cases), how does the algorithm perform on simulated data sets for which you know the "truth"? 

## Applications to real data sets

Test the algorithm on the real-world examples in the orignal paper if possible. Try to find at least one other real-world data set not in the original paper and test it on that. Describe and interpret the results.

## Comparative analysis with competing algorihtms

Find two other algorihtms that addresss a similar problem. Perform a comparison - for example, of accurary or speed. You can use native libraires of the other algorithms - you do not need to code them yourself. Comment on your observations.

## Discussion/conclusion

Your thoughts on the algorithm. Does it fulfill a particular need? How could it be generalized to other problem domains? What are its limiations and how could it be improved further?

## References/bibliography

Make sure you cite your sources.